L'objectif de ce notebook était de trouver le meilleur pre-processing possible afin de permettre la détection de texte grâce à la bibliothèque pytesseract. Les résultats sont peu pertinents.

In [168]:
#Import des bibliothèques
import cv2
import re
import glob
import shutil
import numpy as np
import sys
from PIL import Image,ImageEnhance, ImageFilter
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [112]:
#Détection du sens de 'limage à partir du centre de gravité d'un objet (peu probant)
image = cv2.imread("D:/Users/leoca/Documents/Cours/Projet_3A/dataset-20230207T204629Z-001/dataset/ingredients/ing_20.jpg", cv2.IMREAD_COLOR)

if image.size == 0:
  sys.exit("Error: the image has not been correctly loaded.")

gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

_,binary = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)

contours,hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
CountersImg = max(contours,key = cv2.contourArea)
#print(CountersImg)
moments = cv2.moments(CountersImg)
cx = int(moments["m10"]/moments["m00"])
cy = int(moments["m01"]/moments["m00"])
height, width = image.shape[:2]
center_x = width//2
center_y = height//2

if cx > center_x:
  direction = "right"
elif cx < center_x:
  direction = "left"
if cy > center_y :
  direction = "down"
else:
  direction = "up"

print(direction)

up


In [131]:
#Import de l'image avec la bibliothèque CV2
simage = r"D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition/nut_104.jpg"
img = cv2.imread(simage)
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [157]:
#Import de l'image avec la bibliothèque PILLOW
img = Image.open("D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition/nut_501.jpg")
# Filtrage (augmentation du contraste)
#im = img.filter(ImageFilter.MedianFilter())
#enhancer = ImageEnhance.Contrast(im)
#im = enhancer.enhance(0.1)
#im = im.convert('1')

In [164]:
#Reconnaissance de caractères avec pytesseract suite à l'importation PILLOW
def characters_recognition_PIL(img):
  count=0
  rotated_images = [img,img.transpose(Image.Transpose.ROTATE_90), img.transpose(Image.Transpose.ROTATE_180), img.transpose(Image.Transpose.ROTATE_270)]
  for image in rotated_images:
    text = pytesseract.image_to_string(image, lang ='fra')
    text2 = pytesseract.image_to_string(image, lang ='eng')
    NUTRITION = re.compile(".UTRITION.")
    INFORMATION = re.compile(".INFORMATION.")
    nutrition = re.compile(".utrition.")
    information = re.compile(".nformation.")
    ENERGIE = re.compile(".NERGIE.")
    energie = re.compile(".nergie.")
    valeurs = re.compile(".aleur.")
    count+=1
    if NUTRITION.search(text) or INFORMATION.search(text) or nutrition.search(text) or information.search(text) or valeurs.search(text) or ENERGIE.search(text) or energie.search(text) or NUTRITION.search(text2) or INFORMATION.search(text2) or nutrition.search(text2) or information.search(text2) or valeurs.search(text2) or ENERGIE.search(text2) or energie.search(text2):
      print("présent",count)
      return True
      break

  return False
characters_recognition_PIL(img)

présent 1


True

In [122]:
#Application d'un threshold
threshold=150 #value above which pixel values will be set to max_value
max_value=255 #value to which pixels above threshold will be set
threshold_stype=cv2.THRESH_BINARY #default threshold method
ret, img_binary = cv2.threshold(img_gray, threshold, max_value, threshold_stype)

In [132]:
#Reconnaissance de caractères avec pytesseract suite à l'importation CV2
def characters_recognition(image):
  rotated_images = [cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE), cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE), cv2.rotate(image, cv2.ROTATE_180)]
  associated_texts = []
  len_texts = []

  for image in rotated_images:
    text = pytesseract.image_to_string(image, lang='fra')
    print(text)
    associated_texts.append(text)
    len_texts.append(len(text))

  return associated_texts[np.argmax(len_texts)]

#Filtre de bruit pour essayer d'améliorer la détection
def remove_noise(image):
  anoise_image = cv2.medianBlur(image,5)
  #print(np.shape(anoise_image))
  return anoise_image


In [133]:
characters_recognition(img_gray)

NUTRITIONAL

INFORMATION:

Typical Values per 100g.

Energy 1050K
284kcal

Protein 0.2g

Carbohydrate 635

Fat

Trace

9OEL

E
259 S11PAYoquez
270 UI9)01g
1PNF8C
PMOSOT KBioug

”A001 19d sonjez |eoidép
‘NOLLVAMOANI

TWYNOLLIMLAN

9OEL

E
259 S11PAYoquez
270 UI9)01g
1PNF8C
PMOSOT KBioug

”A001 19d sonjez |eoidép
‘NOLLVAMOANI

TWYNOLLIMLAN



'NUTRITIONAL\n\nINFORMATION:\n\nTypical Values per 100g.\n\nEnergy 1050K\n284kcal\n\nProtein 0.2g\n\nCarbohydrate 635\n\nFat\n\nTrace\n'

In [169]:
#Main : boucle sur l'ensemble du set nutrition (440 images)
images_nutrition =[]
for filename in glob.glob('D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition/*.jpg'):
     im=Image.open(filename)
     bool = characters_recognition_PIL(im)
     if bool==True:
          shutil.move(filename, 'D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition_classe')
          images_nutrition.append(filename)
          print(filename)

présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_101.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_114.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_12.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_1237.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_13.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_152.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_156.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_16.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_1622.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_1645.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_1683.jpg
présent 1
D:/Users/leoca/Documents/Cours/Projet_3A/dataset/nutrition\nut_1698.jpg
présent 4
D:/Users/leoca/D